In [1]:
import os

import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

from sklearn.model_selection import train_test_split


/opt/miniconda3/lib/python3.13/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


## Data Loading

In [2]:
path = '~/Desktop/CareerFoundry/3.1./'
y = pd.read_csv(os.path.join(path, 'Data/Original/Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'))
X = pd.read_pickle(os.path.join(path, 'Data/Clean/cleaned_for_keras.pkl'))

In [3]:
y.drop(columns='DATE', inplace=True)

In [4]:

X = np.array(X)
y = np.array(y)

In [5]:
X = X.reshape(-1,15,9)
X.shape, y.shape

((22950, 15, 9), (22950, 15))

## Train/Test Split

In [6]:

X_train, X_test, y_train, y_test = train_test_split(
    X,y,random_state = 42
)

In [7]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(17212, 15, 9) (17212, 15)
(5738, 15, 9) (5738, 15)


## RNN Architecture Design

In [8]:
model = Sequential()

model.add(
    LSTM(
        units=64,
        input_shape=(15, 9)
    )
)

model.add(Dense(64, activation='relu'))

# Multi-label output
model.add(Dense(15, activation='sigmoid'))

model.summary()


/opt/miniconda3/lib/python3.13/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        18,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │           975 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,079 (94.06 KB)

 Trainable params: 24,079 (94.06 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [10]:
history = model.fit(
    X_train,
    y_train,
    epochs=30,
    batch_size=16,
    validation_split=0.2,
    verbose=2
)


Epoch 1/30
861/861 - 4s - 4ms/step - accuracy: 0.1174 - loss: 0.2784 - val_accuracy: 0.1592 - val_loss: 0.2442
Epoch 2/30
861/861 - 2s - 3ms/step - accuracy: 0.1609 - loss: 0.2340 - val_accuracy: 0.1429 - val_loss: 0.2209
Epoch 3/30
861/861 - 2s - 3ms/step - accuracy: 0.1652 - loss: 0.2179 - val_accuracy: 0.1818 - val_loss: 0.2058
Epoch 4/30
861/861 - 2s - 3ms/step - accuracy: 0.1723 - loss: 0.2017 - val_accuracy: 0.1705 - val_loss: 0.1930
Epoch 5/30
861/861 - 2s - 3ms/step - accuracy: 0.1860 - loss: 0.1889 - val_accuracy: 0.1952 - val_loss: 0.1789
Epoch 6/30
861/861 - 2s - 3ms/step - accuracy: 0.1923 - loss: 0.1765 - val_accuracy: 0.2103 - val_loss: 0.1758
Epoch 7/30
861/861 - 2s - 3ms/step - accuracy: 0.1917 - loss: 0.1672 - val_accuracy: 0.2170 - val_loss: 0.1597
Epoch 8/30
861/861 - 2s - 3ms/step - accuracy: 0.2002 - loss: 0.1582 - val_accuracy: 0.2077 - val_loss: 0.1615
Epoch 9/30
861/861 - 2s - 3ms/step - accuracy: 0.2024 - loss: 0.1505 - val_accuracy: 0.1900 - val_loss: 0.1532
E

In [11]:
y_pred = (model.predict(X_test) > 0.5).astype(int)

overall_accuracy = (y_pred == y_test).mean()
station_accuracy = (y_pred == y_test).mean(axis=0)

overall_accuracy, station_accuracy


180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


(np.float64(0.9828744045544324),
 array([0.98710352, 0.95451377, 0.98658069, 0.98170094, 0.96183339,
        0.9654932 , 0.98170094, 0.99006623, 0.98605786, 0.98849773,
        0.99337748, 0.99128616, 1.        , 0.98518648, 0.98971767]))

In [12]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'

}

In [13]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [14]:
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 989us/step
Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL         296       585        49      71          26       218      11   
BELGRADE        0       199        65      18           4        75       5   
BUDAPEST        0         0        37       0           0        41       2   
DEBILT          0         0         0       6           6        19       0   
DUSSELDORF      0         0         0       0           0         7       0   
HEATHROW        0         0         0       0           1        36       0   
KASSEL          0         0         0       0           0         0       2   
LJUBLJANA       0         0         0       0           0         0       0   
MAASTRICHT      0         0         0       0           1         1       0   
MADRID          0         0         0       0           0         0       0   
MUNCHENB 

Low and unstable accuracy suggested underfitting. Increasing the number of LSTM units increases the model’s ability to capture temporal dependencies.

Adjustments

In [15]:
model = Sequential()

model.add(
    LSTM(
        units=32,
        input_shape=(15, 9)
    )
)

model.add(Dense(64, activation='relu'))

# Multi-label output
model.add(Dense(15, activation='sigmoid'))

model.summary()


/opt/miniconda3/lib/python3.13/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 32)             │         5,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 15)             │           975 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,463 (33.06 KB)

 Trainable params: 8,463 (33.06 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [ ]:
history = model.fit(
    X_train,
    y_train,
    epochs=30,
    batch_size=32, ## changed
    validation_split=0.2,
    verbose=2
)


Epoch 1/30


In [ ]:
y_pred = (model.predict(X_test) > 0.5).astype(int)

overall_accuracy = (y_pred == y_test).mean()
station_accuracy = (y_pred == y_test).mean(axis=0)

overall_accuracy, station_accuracy


180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


(np.float64(0.9400604159405135),
 array([0.95834786, 0.92366678, 0.89874521, 0.92418961, 0.93342628,
        0.87155803, 0.92331823, 0.90275357, 0.93429766, 0.94109446,
        0.96653886, 0.97385849, 1.        , 0.96950157, 0.97960962]))

In [ ]:
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 711us/step
Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL         356      1735        47       0         143        91       2   
BELGRADE        6       468         4       1          24         0       0   
BUDAPEST        4        33         3       0           8         1       0   
DEBILT          0         0         0       0          10         0       0   
DUSSELDORF      0         0         0       0           2         0       0   
HEATHROW        2         1         0       0           1         7       0   
KASSEL          1         0         0       0           1         0       0   
LJUBLJANA       2         4         2       0           0         1       0   
MAASTRICHT      0         0         1       0           0         1       0   
MADRID          3         1         1       0           0         2       0   
MUNCHENB 

Adjustments

In [ ]:
model = Sequential()

model.add(
    LSTM(
        units=32, ## changed
        input_shape=(15, 9)
    )
)

model.add(Dense(64, activation='relu'))

# Multi-label output
model.add(Dense(15, activation='sigmoid'))

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 32)             │         5,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 15)             │           975 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,463 (33.06 KB)

 Trainable params: 8,463 (33.06 KB)

 Non-trainable params: 0 (0.00 B)

The LSTM model consists of a single recurrent layer followed by two dense layers.
Compared to the CNN architecture, the LSTM contains fewer trainable parameters,
reflecting a more compact model design.

The LSTM layer processes the sequence of weather stations as ordered inputs,
producing a fixed-length representation that is subsequently mapped to
station-level pleasant weather predictions via sigmoid-activated output units.

Compile and Run

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [ ]:
history = model.fit(
    X_train,
    y_train,
    epochs=30,
    batch_size=16, ## chnaged
    validation_split=0.2,
    verbose=2
)


Epoch 1/30
861/861 - 3s - 4ms/step - accuracy: 0.0905 - loss: 0.2903 - val_accuracy: 0.1281 - val_loss: 0.2484
Epoch 2/30
861/861 - 2s - 2ms/step - accuracy: 0.1495 - loss: 0.2426 - val_accuracy: 0.1690 - val_loss: 0.2310
Epoch 3/30
861/861 - 2s - 2ms/step - accuracy: 0.1901 - loss: 0.2269 - val_accuracy: 0.1775 - val_loss: 0.2200
Epoch 4/30
861/861 - 2s - 2ms/step - accuracy: 0.2169 - loss: 0.2169 - val_accuracy: 0.1992 - val_loss: 0.2101
Epoch 5/30
861/861 - 2s - 2ms/step - accuracy: 0.2527 - loss: 0.2071 - val_accuracy: 0.1807 - val_loss: 0.2032
Epoch 6/30
861/861 - 2s - 2ms/step - accuracy: 0.2755 - loss: 0.2000 - val_accuracy: 0.2248 - val_loss: 0.2024
Epoch 7/30
861/861 - 2s - 2ms/step - accuracy: 0.2824 - loss: 0.1933 - val_accuracy: 0.2902 - val_loss: 0.1892
Epoch 8/30
861/861 - 2s - 2ms/step - accuracy: 0.2847 - loss: 0.1863 - val_accuracy: 0.3230 - val_loss: 0.1850
Epoch 9/30
861/861 - 2s - 2ms/step - accuracy: 0.2849 - loss: 0.1805 - val_accuracy: 0.3413 - val_loss: 0.1867
E

The training process exhibits moderate instability, with accuracy peaking early
and subsequently declining despite continued loss reduction.

This behavior suggests that the LSTM struggles to extract consistent predictive
patterns from the imposed station sequence. While the model continues to improve
probability calibration (as reflected by decreasing loss), these improvements do
not consistently translate into better thresholded predictions.


In [ ]:
y_pred = (model.predict(X_test) > 0.5).astype(int)

overall_accuracy = (y_pred == y_test).mean()
station_accuracy = (y_pred == y_test).mean(axis=0)

overall_accuracy, station_accuracy


180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


(np.float64(0.9554548623213663),
 array([0.97385849, 0.952771  , 0.93900314, 0.93116068, 0.93900314,
        0.87486929, 0.92471244, 0.94701987, 0.93534333, 0.97664692,
        0.98640641, 0.98466365, 1.        , 0.97804113, 0.98832346]))

overall_accuracy = 0.9554548623213663
The overall label-wise accuracy of the LSTM model is high; however, this result is
strongly influenced by the prevalence of negative labels in the dataset.

Compared to the CNN, the LSTM achieves slightly lower overall accuracy, indicating
reduced effectiveness in distinguishing station-level pleasant weather outcomes.


Station-level accuracy varies more substantially for the LSTM model than for the
CNN. This uneven performance suggests that the sequential modeling assumption
does not benefit all stations equally and may introduce noise rather than useful
structure for certain locations.

In [ ]:
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step
Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL         184       492       327       0         177       417      11   
BELGRADE        3       282         7       0           9         0       0   
BUDAPEST        1         9        14       0           5         2       1   
DEBILT          0         1         0       1           4         0       1   
DUSSELDORF      0         0         0       0           3         0       0   
HEATHROW        0         1         0       0           4         3       0   
KASSEL          1         0         0       0           0         0       2   
LJUBLJANA       0         1         1       0           0         0       0   
MAASTRICHT      0         0         0       0           1         2       0   
MADRID          0         0         0       0           0         0       0   
MUNCHENB 

The confusion matrix was generated by applying an argmax operation across station
outputs, which implicitly converts the multi-label problem into a multi-class
setting. As a result, the matrix does not represent true classification errors
and should be interpreted with caution.

The dominance of certain stations in the matrix reflects relative signal strength
rather than meaningful station-level confusion. For this reason, label-wise
accuracy remains a more appropriate evaluation metric for this task.
